In [1]:
import cv2
import numpy as np
import json

# COLORS
RGB_black = (0,0,0)
RGB_red = (0,0,255)
RGB_green = (0,255,0)
RGB_blue = (255,0,0)
RGB_orange = (0, 165, 255)
RGB_purple = (221,160,221)
RGB_gray = (220,220,220)


##################################################
LB_json_file = "datasetRender.json"                 # json created from LB2VIA_extract
image_directory = "handrail_calib_1.jpg"            # caputred image

#.# 
img = cv2.imread(image_directory)
h_img,w_img,channels = img.shape

### PLOT POLYGON LINES ###

with open(LB_json_file, 'r') as f:
    via_dict = json.load(f)
    
# create empty dictionary to serve as 
img_file_name = "ckxxukz4fvvix0z961wi3gd94.jpg690284"                                   # NEED TO AUTOMATE FINDING THIS VALUE
x_points = via_dict[img_file_name]['regions']['0']['shape_attributes']['all_points_x']
y_points = via_dict[img_file_name]['regions']['0']['shape_attributes']['all_points_y']

# print(np.array(x_points).shape)
# print("x_points: ",x_points)

# print(np.array(x_points).shape)
# print("y_points: ",y_points)

poly_points = []
for i in range(len(x_points)):
    poly_points.append([x_points[i],y_points[i]])

poly_points = np.array(poly_points, np.int32)
poly_points = poly_points.reshape((-1,1,2))
img = cv2.polylines(img,[poly_points],True,RGB_red,4)
# print(poly_points)
# print(poly_points.shape)
x1,y1 = 46,167
x2,y2 = 1062,167
x_mid = int(np.average([x1,x2]))
y_mid = int(np.average([y1,y2]))
i = 1
for i in range(poly_points.shape[0]-1):
    x1 = poly_points[i][0][0]
    y1 = poly_points[i][0][1]
    x2 = poly_points[i+1][0][0]
    y2 = poly_points[i+1][0][1]
    x_mid = int(np.average([x1,x2]))
    y_mid = int(np.average([y1,y2]))
    img = cv2.putText (img, str(i), (x_mid,y_mid), cv2.FONT_HERSHEY_SIMPLEX, 0.5, RGB_black, 2)


print("Poly Shape: ",poly_points.shape)

### Display Image ###
scale_percent = 100 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

# cv2.imshow("handrail.jpg", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()



Poly Shape:  (129, 1, 2)


In [92]:
def DotProduct(a,b):
    """
    Find the Dot Product between vectors a and b
    Input: vector a, vector b
    Output: angle ans"""
    ans = np.arccos((np.dot(a,b) / (np.linalg.norm(a)*np.linalg.norm(b)))) *180/np.pi
    return ans

def Dot2Rect(x,y):  
    """
    Find pixel points around a coordinate of interest 
    for visualizing pixel placement in image
    Input: coordinates of interst
    Output: min and max x,y to define rectangle"""
    d = 4
    x_min = int(x-d)
    y_min = int(y-d)
    x_max = int(x+d)
    y_max = int(y+d)
    return x_min,y_min,x_max,y_max


# DEFINE INTERVAL PARAMETERS
num_pix_per_interval = 3                # at first, set to 1
pix2pix_dist = 15                        # at first, set to 1

# Subcalcs
num_pix_per_arm = int((num_pix_per_interval-1)/2)


# LOI = [0,78]
LOI = np.linspace(0,len(poly_points)-1,50, dtype=int)

for i in range(len(LOI)):
    print("LINE OF INTEREST: ", LOI[i])
    x1 = poly_points[LOI[i]][0][0]
    y1 = poly_points[LOI[i]][0][1]
    
    if LOI[i] == len(poly_points)-1:
        x2 = poly_points[LOI[0]][0][0]
        y2 = poly_points[LOI[0]][0][1]
    else:
        x2 = poly_points[LOI[i]+1][0][0]
        y2 = poly_points[LOI[i]+1][0][1]

    vx,vy = x2-x1,y2-y1

    quad1,quad2,quad3,quad4 = False,False,False,False
    if(vx>=0 and vy<=0):
        quad1 = True
        # print("QUAD 1")
    elif(vx>=0 and vy>=0):
        quad2 = True
        # print("QUAD 2")
    elif(vx<=0 and vy>=0):
        quad3 = True
        # print("QUAD 3")
    elif(vx<=0 and vy<=0):
        quad4 = True
        # print("QUAD 4")

    a = [1,0]                       # with resect to the width of the image
    b = [vx,vy]

    # find the angle THETA1 of the line segment wrt image horizon
    # find the angle THETA2 of a line tangent to
    if quad1:
        theta1 = DotProduct(a,b)
        theta2 = -theta1 + 90
    elif quad2:
        theta1 = DotProduct(a,b)
        theta2 = -theta1 + 90
    elif quad3:
        theta1 = DotProduct(a,b) 
        theta2 = theta1 - 90
    elif quad4:
        theta1 = DotProduct(a,b) 
        theta2 = theta1 - 90

    # print("t1: ", theta1)
    # print("t2: ", theta2)
    theta2 = theta2*np.pi/180

    # CENTER POINT IS TEMPORARY FOR ADDING >1 INTERVALS PER LINE
    xc,yc = (x1+x2)/2, (y1+y2)/2
    # print("Center Point: ", [xc,yc])

    
    
    xc_c,yc_c = [],[]
    xc_cc,yc_cc = [],[]
    for i in range(num_pix_per_arm):

        # theta2 = 90*np.pi/180
        if(quad1):
            # counter clockwise pixels extending from tangent line
            xc_c.append(xc - (i+1)*pix2pix_dist*np.cos(theta2))
            yc_c.append(yc - (i+1)*pix2pix_dist*np.sin(theta2))

            # clockwise pixels extending from tangent line
            xc_cc.append(xc + (i+1)*pix2pix_dist*np.cos(theta2))
            yc_cc.append(yc + (i+1)*pix2pix_dist*np.sin(theta2))
        elif(quad2):
            # counter clockwise pixels extending from tangent line
            xc_c.append(xc + (i+1)*pix2pix_dist*np.cos(theta2))
            yc_c.append(yc - (i+1)*pix2pix_dist*np.sin(theta2))

            # clockwise pixels extending from tangent line
            xc_cc.append(xc - (i+1)*pix2pix_dist*np.cos(theta2))
            yc_cc.append(yc + (i+1)*pix2pix_dist*np.sin(theta2))
        elif(quad3):
            # counter clockwise pixels extending from tangent line
            xc_c.append(xc + (i+1)*pix2pix_dist*np.cos(theta2))
            yc_c.append(yc + (i+1)*pix2pix_dist*np.sin(theta2))

            # clockwise pixels extending from tangent line
            xc_cc.append(xc - (i+1)*pix2pix_dist*np.cos(theta2))
            yc_cc.append(yc - (i+1)*pix2pix_dist*np.sin(theta2))
        elif(quad4):
            # counter clockwise pixels extending from tangent line
            xc_c.append(xc - (i+1)*pix2pix_dist*np.cos(theta2))
            yc_c.append(yc + (i+1)*pix2pix_dist*np.sin(theta2))

            # clockwise pixels extending from tangent line
            xc_cc.append(xc + (i+1)*pix2pix_dist*np.cos(theta2))
            yc_cc.append(yc - (i+1)*pix2pix_dist*np.sin(theta2))



    # print("pseudo clockwise [xc_c,yc_c]: ", [xc_c,yc_c])
    # print("pseudo counterclockwise [xc_cc,yc_cc]: ", [xc_cc,yc_cc])

    #interweve  x,y coordinates s.t. (x1,y1)...(xn,yn)

    xy_c = []
    xy_cc = []
    for i in range(len(xc_c)):
        
        xy_c.append([xc_c[i],yc_c[i]])
        xy_cc.append([xc_cc[i],yc_cc[i]])

    # print("pseudo clockwise interwoven [xc_c[i],yc_c[i]]: ", xy_c)
    # print("pseudo counterclockwise interwoven [xc_cc[i],yc_cc[i]]: ", xy_cc)
    xy_c.reverse()

    xy_interval = []
    for i in range(num_pix_per_arm):
        xy_interval.append(xy_c[i])

    xy_interval.append([xc,yc])

    for i in range(num_pix_per_arm):
        xy_interval.append(xy_cc[i])


    pts_int = np.array([xy_interval[0],xy_interval[-1]], np.int32)
    pts_int = pts_int.reshape((-1,1,2))
    # print("Poly Shape: ",pts.shape)
    img = cv2.polylines(img,[pts_int],True,RGB_green,4)

    idx = 0
    for i in xy_interval:
        idx = idx + 1
        temp_var = idx%num_pix_per_interval
        x_c_min,y_c_min,x_c_max,y_c_max = Dot2Rect(i[0],i[1])

        if(temp_var==1):
            temp_color = RGB_orange
        elif (temp_var==0):
            temp_color = RGB_purple
        else:
            temp_color = RGB_gray
        img = cv2.rectangle(img, (x_c_min,y_c_min), (x_c_max,y_c_max), temp_color, -1)


    
    BGR_interval = []
    orig_img = cv2.imread(image_directory)
    
    xy_interval = np.array(xy_interval, dtype=int)
    xy_interval = np.ndarray.tolist(xy_interval)
    for i in range(len(xy_interval)):
        x_BGR, y_BGR = int(xy_interval[i][0]),int(xy_interval[i][1])
        B,G,R = orig_img[y_BGR,x_BGR]
        BGR_interval.append([B,G,R])
    
    print("xy_interval: ", xy_interval)         # interval coordinates
    print("BGR list: ", BGR_interval)           # interval BRG values
    

# cv2.imshow("handrail.jpg", img)             # show interval of interest on edited image

# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite("handrail.jpg", img)

LINE OF INTEREST:  0
xy_interval:  [[554, 152], [554, 167], [554, 182]]
BGR list:  [[255, 255, 255], [234, 253, 255], [90, 221, 240]]
LINE OF INTEREST:  2
xy_interval:  [[1079, 177], [1064, 179], [1049, 181]]
BGR list:  [[255, 255, 255], [203, 237, 243], [77, 203, 227]]
LINE OF INTEREST:  5
xy_interval:  [[1079, 201], [1064, 203], [1049, 205]]
BGR list:  [[255, 255, 255], [185, 234, 244], [72, 179, 212]]
LINE OF INTEREST:  7
xy_interval:  [[1081, 212], [1066, 212], [1051, 212]]
BGR list:  [[255, 255, 255], [207, 238, 247], [81, 195, 225]]
LINE OF INTEREST:  10
xy_interval:  [[1084, 229], [1069, 229], [1054, 229]]
BGR list:  [[255, 255, 255], [223, 249, 255], [85, 204, 230]]
LINE OF INTEREST:  13
xy_interval:  [[1086, 244], [1071, 244], [1056, 244]]
BGR list:  [[255, 255, 255], [214, 244, 255], [90, 203, 229]]
LINE OF INTEREST:  15
xy_interval:  [[1085, 242], [1072, 249], [1059, 255]]
BGR list:  [[255, 254, 255], [196, 239, 248], [85, 208, 234]]
LINE OF INTEREST:  18
xy_interval:  [[109

True

In [93]:
# # CUSTOM NUMBER OF INTERVALS
# num_intervals = 1                            # number of verital intervals (i.e. y1=y2)
# mids_x = np.linspace(x1,x2,num_intervals+2,dtype=int) 
# mids_y = np.linspace(y1,y2,num_intervals+2,dtype=int)    

# print(mids_x)
# print(mids_y)

# mids_x = np.delete(mids_x,0)
# mids_x = np.delete(mids_x,-1)
# mids_y = np.delete(mids_y,0)
# mids_y = np.delete(mids_y,-1)

# print(mids_x)
# print(mids_y)

In [94]:
# # PIXEL LEVEL ANALYSIS
# i = 4           # analyze pixel in interval
# x_BGR, y_BGR = int(xy_interval[i][0]),int(xy_interval[i][1])
# B,G,R = orig_img[y_BGR,x_BGR]

# COLOR PICKER TEST
# orig_img = cv2.imread("Flag.jpg")
# x_BGR, y_BGR = 200,200

# x_c_min,y_c_min,x_c_max,y_c_max = Dot2Rect(x_BGR, y_BGR)
# orig_img = cv2.rectangle(orig_img, (x_c_min,y_c_min), (x_c_max,y_c_max), (RGB_black), -1)

# print("coorRGB: ", [x_BGR, y_BGR])          # pixel coordinates
# print ("BGR value: ", [B,G,R])              # pixel BGR values


# cv2.imshow("handrail.jpg", orig_img)        # show pixel of interest on original image
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [95]:


### PLOT INTERVALS BETWEEN 2 POINTS ###
# LOI = [0,78]

# x1 = poly_points[LOI[0]][0][0]
# y1 = poly_points[LOI[0]][0][1]
# x2 = poly_points[LOI[0]+1][0][0]
# y2 = poly_points[LOI[0]+1][0][1]


# pts = np.array([[x1,y1],[x2,y2]], np.int32)
# pts = pts.reshape((-1,1,2))
# # print("Poly Shape: ",pts.shape)
# img = cv2.polylines(img,[pts],True,RGB_blue,4)

# num_vert_int = 1                            # number of verital intervals (i.e. y1=y2)
# mids = np.linspace(x1,x2,num_vert_int+2,dtype=int)    
# print(mids)
# mids = np.delete(mids,0)
# mids = np.delete(mids,-1)
# print(mids)

# num_pix_per_int = 3                         # number of pixels per interval (MUST BE ODD)
# pix_sep = 10

# pix_coor = []
# for i in range(len(mids)):
#     print(mids[i])


# cv2.imshow("handrail.jpg", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


129